In [62]:
import requests
import pandas as pd

# Step 1: Read the GitHub token from the file
with open('../github_token.txt', 'r') as file:
    github_token = file.read().strip()

headers = {'Authorization': f'token {gi}thub_token}'}

In [63]:
#getting the user data
search_url = 'https://api.github.com/search/users?q=location:Sydney+followers:>100'
response = requests.get(search_url, headers=headers)

# Check if request was successful
if response.status_code == 200:
    users_data = response.json()
    
    # Step 3: Initialize an empty list to store user details
    users_list = []

    # Loop through each user returned by the search
    for user in users_data['items']:
        user_api_url = user['url']  # Get detailed user API URL
        
        # Step 4: Request detailed user data using the user's API URL
        user_response = requests.get(user_api_url, headers=headers)
        
        if user_response.status_code == 200:
            user_details = user_response.json()

            # Extract the necessary information, and apply cleaning as required
            user_info = {
                'login': user_details.get('login', ''),
                'name': user_details.get('name', ''),
                'company': user_details.get('company', '').lstrip('@').strip().upper() if user_details.get('company') else '',
                'location': user_details.get('location', ''),
                'email': user_details.get('email', ''),
                'hireable': user_details.get('hireable', False),
                'bio': user_details.get('bio', ''),
                'public_repos': user_details.get('public_repos', 0),
                'followers': user_details.get('followers', 0),
                'following': user_details.get('following', 0),
                'created_at': user_details.get('created_at', '')
            }

            # Step 5: Append the cleaned user data to the list
            users_list.append(user_info)
        
        else:
            print(f"Failed to retrieve details for user {user['login']}. Status code: {user_response.status_code}")

    # Step 6: Create a DataFrame and save to CSV
    users_df = pd.DataFrame(users_list)
    print(len(users_df))
    #users_df.to_csv('users.csv', index=False)

    #print("Data successfully saved to 'users.csv'.")

else:
    print(f"Failed to retrieve users. Status code: {response.status_code}")

30


In [61]:
repo_details = []
#getting the repo data 
users = users_data['items']
for repos in users:
    repo = repos['repos_url']
    repo_data = requests.get(repo,headers=headers)
    datum = repo_data.json()
    for data in datum[:500]:
        repo_info = {
                'login': data.get('owner', '')['login'],
                'name': data.get('full_name', ''),
                'created_at': data.get('created_at', ''),
                'stargazers_count': data.get('stargazers_count', ''),
                'watchers_count': data.get('watchers_count', ''),
                'language': data.get('language', ''),
                'has_projects': data.get('has_projects', ''),
                'has_wiki': data.get('has_wiki', ''),
                'license_name': data['license']['name'] if data.get('license') is not None else ''
            }
        repo_details.append(repo_info)
repo_df = pd.DataFrame(repo_details)
repo_df.to_csv('repo_df.csv', index=False)
print('completed')

completed


completed
